In [142]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [143]:
import numpy as np

num_classes = 10
input_shape = (28, 28, 1)

X_train = X_train.astype("float32") / 255.
X_test = X_test.astype("float32") / 255.

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)
print("X_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [52]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

Not connected to a TPU runtime. Using CPU/GPU strategy
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [160]:
from tf.keras import layers

with strategy.scope():
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="elu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)               

In [161]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 47s 108ms/step - loss: 0.3612 - accuracy: 0.8940 - val_loss: 0.0890 - val_accuracy: 0.9775
Epoch 2/15
422/422 [==============================] - 44s 105ms/step - loss: 0.1124 - accuracy: 0.9659 - val_loss: 0.0648 - val_accuracy: 0.9838
Epoch 3/15
422/422 [==============================] - 44s 105ms/step - loss: 0.0845 - accuracy: 0.9739 - val_loss: 0.0561 - val_accuracy: 0.9845
Epoch 4/15
422/422 [==============================] - 44s 105ms/step - loss: 0.0736 - accuracy: 0.9768 - val_loss: 0.0503 - val_accuracy: 0.9863
Epoch 5/15
422/422 [==============================] - 44s 105ms/step - loss: 0.0663 - accuracy: 0.9793 - val_loss: 0.0489 - val_accuracy: 0.9865
Epoch 6/15
422/422 [==============================] - 44s 104ms/step - loss: 0.0602 - accuracy: 0.9815 - val_loss: 0.0439 - val_accuracy: 0.9882
Epoch 7/15
422/422 [==============================] - 44s 104ms/step - loss: 0.0562 - accuracy: 0.9818 - val_loss: 0.0474 - val_ac

In [165]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.02930765599012375
Test accuracy: 0.9900000095367432
